In [39]:
import math
import operator
import numpy as np
import pandas as pd


def calculateFunc(number, x):
    if number == 1:
        return -5 * x ** 5 + 4 * x  ** 4 - 12 * x ** 3 + 11 * x ** 2 - 2 * x + 1 
    elif number == 2:
        return -3 * x * np.sin(0.75 * x) + np.exp(-2 * x)
    elif number == 3:
        return np.exp(3 * x) + 5 * np.exp(-2 * x)


def calculateDeriv(number, x):
    if number == 1:
        return -25 * x ** 4 + 16 * x ** 3 - 36 * x ** 2 + 22 * x - 2
    elif number == 2:
        return -2 * np.exp(-2 * x) - 3 * np.sin(0.75 * x) - 2.25 * x * np.cos(0.75 * x)
    elif number == 3:
        return np.exp(-2 * x) * (3 * np.exp(5 * x) - 10)


def calculateX_hat(x1, x2, f1, f2):
    a = (f2 - f1) / (x2 - x1)
    b = f1 - x1 * a
    return - (b / a)


def brent(a, c, eps, func_index):
    data = {'a' : [], 'c' : [], 'x' : [], 'w' : [], 'v' : [],
            'u': [], 'f_x' : [], 'f_w' : [], 'f_v' : [], 'f_u' : []} 
    x = w = v = (a + c) / 2
    f_x = f_w = f_v = calculateFunc(func_index, x)
    f_x_deriv = f_w_deriv = f_v_deriv = calculateDeriv(func_index, x)
    d = e = c - a
    counter = 0
    while True:
        counter += 1
        g, e = e, d     
        u = None
        if x != w and f_x_deriv != f_w_deriv:
            u = calculateX_hat(x, w, f_x_deriv, f_w_deriv)
            if u >= a + eps and u <= c - eps and abs(u - x) < g / 2:
                u = u
            else: 
                u = None
        u2 = None
        if x != v and f_x_deriv != f_v_deriv:
            u2 = calculateX_hat(x, v, f_x_deriv, f_v_deriv)
            if u2 >= a + eps and u2 <= c - eps and abs(u2 - x) < g / 2:
                if u is not None and abs(u2 -x) < abs(u - x):
                    u = u2
        if u is None:
            if f_x_deriv > 0:
                u = (a + x) / 2
            else:
                u = (x + c) / 2
        if abs(u - x) < eps:
            u = x + np.sign(u - x) * eps
        d = abs(x -u)
        f_u = calculateFunc(func_index, u)
        f_u_deriv = calculateDeriv(func_index, u)
        writeRowBrent(data, a, c, x, w, v, u, f_x, f_w, f_v, f_u)
        if f_u <= f_x:
            if u >= x:
                a = x
            else:
                c = x
            v, w, x = w, x, u 
            f_v, f_w, f_x = f_w, f_x, f_u
            f_v_deriv, f_w_deriv, f_x_deriv = f_w_deriv, f_x_deriv, f_u_deriv
        else:
            if u >= x:
                c = u
            else:
                a = u
            if f_u <= f_w or w == x:
                v, w = w, u 
                f_v, f_w = f_w, f_u
                f_v_deriv, f_w_deriv = f_w_deriv, f_u_deriv
            elif f_u <= f_v or v == x or v == w:
                v = u
                f_v = f_u
                f_v_deriv = f_u_deriv
        if counter != 1:
            if abs(prev_u - u) < eps:
                break
        prev_u = u
    df = pd.DataFrame(data)
    with pd.ExcelWriter(f'brent_{func_index}.xlsx') as writer:
        df.to_excel(writer)
    return counter, (c + a) / 2

In [40]:
numbers = [1, 2, 3]
intervals = [[-0.5, 0.5], [0, np.pi * 2], [0, 1]]

for number, interval in zip(numbers, intervals):
    counter, minimum = brent(interval[0], interval[1], 0.000001, number)
    print(f"Функция: {number}, min: {minimum}")

Функция: 1, min: 0.11067132401450525
Функция: 2, min: 2.705787451575547
Функция: 3, min: 0.24079476763087992
